# Benchmark PyRCN

This notebook accompanies the publication "PyRCN: A Toolbox for Exploration and Application of Reservoir Computing Networks" and serves as a benchmark test for PyRCN and other libraries.

Therefore, we reproduce parts of the publication [[1]](#1), where the stock price return volatility was predicted using Echo State Networks.

We compare the performance of different RCN architectures included in PyRCN with the toolboxes [PyESN](https://github.com/cknd/pyESN) and HP-ELM [[2]](#2) using the [scikit-ELM](https://github.com/akusok/scikit-elm) (skELM) as the interface.

In [ ]:
import pandas as pd
import itertools
from collections import OrderedDict

from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from joblib import dump, load

from pyrcn.extreme_learning_machine import ELMRegressor
from pyrcn.echo_state_network import ESNRegressor
from pyrcn.model_selection import SequentialSearchCV
from skelm import ELMRegressor as SkELMRegressor
import seaborn as sns
import matplotlib.pyplot as plt

from preprocessing import ts2super, compute_average_volatility
from model_selection import PredefinedTrainValidationTestSplit
from adapter import PyESN


%matplotlib inline
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png', 'pdf')
sns.set_theme(context="paper", style="whitegrid", font="serif", font_scale=.8)

## Prepare all datasets

We used the datasets provided by [[1]](#1), namely the stock price volatilities for the three Nasdaq companies Caterpillar (CAT), eBay (EBAY) and Microsoft (MSFT), with each datasets having a length of 2745 days.

In [ ]:
datasets = OrderedDict({"CAT": 0, "EBAY": 1, "MSFT": 2})
data = [None] * len(datasets)

for dataset, k in datasets.items():
    data[k] = pd.read_csv(f"../data/{dataset}.csv")["t0"].to_frame()

The input data was the original time series and the output data the time series shifted by 1, 5 and 21 days, respectively.

In [ ]:
H=21  # pre-trained models available for 1, 5, 21

# ts2super simply returns a dataframe containing the original and shifted by H time series.
df = pd.concat([ts2super(d, 0, H) for d in data])
X = df.iloc[:, 0].values.reshape(-1, 1)
y = df.iloc[:, -1].values.reshape(-1, 1)

df_input = df.iloc[:, 0].to_frame()
df_input["t5"] = df_input.t0.rolling(window=5, min_periods=1).mean()
df_input["t22"] = df_input.t0.rolling(window=22, min_periods=1).mean()
df_input

In [ ]:
df_input_extended = pd.concat([pd.DataFrame(data={"t0": [0], "t5": [0], "t22": [0]}), df_input], ignore_index=False).reset_index(drop=True)
df_input_extended["t0"] = df_input_extended["t0"].shift(periods=-1)
df_input_extended = df_input_extended.dropna()
df_input_extended

Provide a custom train test split for this task, and all scores that we aim to evaluate.

In [ ]:
test_fold = [[k] * len(ts2super(d, 0, H)) for k, d in enumerate(data)]
test_fold = list(itertools.chain.from_iterable(test_fold))

ps = PredefinedTrainValidationTestSplit(test_fold=test_fold, validation=True)
ps_test = PredefinedTrainValidationTestSplit(test_fold=test_fold, validation=False)

scoring = {"MSE": "neg_mean_squared_error", "RMSE": "neg_root_mean_squared_error", "R2": "r2"}

## HAR

As in [[1]](#1), we compared two different input data: The original time series and the Heterogeneous Autoregressive (HAR) feature set [[3]](#3), which expands the one-dimensional time series to three dimensions by adding moving-average filtered representations with filter lengths of five and 22 days, respectively. Since the description of the HAR feature set by [[3]](#3), Equations (4)–(7)), and [[1]](#1) have different temporal contexts for the moving average filter, we compared both implementations. We use the description by [[3]](#3) in the following, because the results were comparable to [[1]](#1), even though not equal.

Each input and target time series was separately scaled to the interval [0, 1] before using it to train and test models. The constants therefore were computed on the training dataset of each fold.

In [ ]:
day_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 1})
week_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 5})
month_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 22})
har_features = FeatureUnion(
    transformer_list=[("day", day_volatility_transformer),
                      ("week", week_volatility_transformer),
                      ("month", month_volatility_transformer)])
har_pipeline = Pipeline(
    steps=[("har_features", har_features),
           ("scaler", MinMaxScaler()),
           ("lstsq", TransformedTargetRegressor(transformer=MinMaxScaler()))])

In [ ]:
try:
    search = load(f'../results/har_grid_h{H}.joblib')
except FileNotFoundError:
    search = GridSearchCV(estimator=har_pipeline, param_grid={}, cv=ps_test,
                          scoring=scoring, refit="R2", return_train_score=True,
                          verbose=10).fit(X, y)
    dump(search, f'../results/har_grid_h{H}.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search.cv_results_['mean_train_R2']}\t{search.cv_results_['mean_test_R2']}\t"
      f"{search.cv_results_['mean_train_MSE']}\t{search.cv_results_['mean_test_MSE']}\t"
      f"{search.cv_results_['mean_fit_time']*1e3}\t{search.cv_results_['mean_score_time']*1e3}")

## PyRCN ESN

Here, we optimize ESNs from PyRCN using only the original time series as features. During the optimization, we only use the training and the validation sets. The test set remains untouched.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. We aimed to jointly optimize input scaling $\alpha_{\mathrm{u}}$ and spectral radius $\rho$, then leakage $\lambda$, bias scaling $\alpha_{\mathrm{bi}}$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyrcn_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4,
        'input_activation': 'identity', 'bias_scaling': 0.0,
        'spectral_radius': 0.0, 'leakage': 1.0, 'k_rec': 10,
        'reservoir_activation': 'tanh', 'bidirectional': False,
        'alpha': 1e-5, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=ESNRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__leakage': uniform(1e-5, 1e0)}
    step3_params = {'esn__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step4_params = {'esn__regressor__hidden_layer_size':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step4 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3),
        ('step4', RandomizedSearchCV, step4_params, kwargs_step4)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyrcn_seq_esn_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyrcn_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyrcn_seq_esn_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

Next, we optimize ESNs from PyRCN using HAR as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. We aimed to jointly optimize input scaling $\alpha_{\mathrm{u}}$ and spectral radius $\rho$, then leakage $\lambda$, bias scaling $\alpha_{\mathrm{bi}}$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyrcn_har_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4,
        'input_activation': 'identity', 'bias_scaling': 0.0,
        'spectral_radius': 0.0, 'leakage': 1.0, 'k_rec': 10,
        'reservoir_activation': 'tanh', 'bidirectional': False,
        'alpha': 1e-5, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=ESNRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__leakage': uniform(1e-5, 1e0)}
    step3_params = {'esn__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step4_params = {'esn__regressor__hidden_layer_size':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step4 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3),
        ('step4', RandomizedSearchCV, step4_params, kwargs_step4)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyrcn_har_seq_esn_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyrcn_har_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyrcn_har_seq_esn_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

## PyRCN ELM

Next, we optimize ELMs from PyRCN using only the original time series as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. We aimed to optimize input scaling $\alpha_{\mathrm{u}}$, then bias scaling $\alpha_{\mathrm{bi}}$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyrcn_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4,
        'input_activation': 'tanh', 'bias_scaling': 0.0,
        'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("elm", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_elm_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'elm__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyrcn_seq_elm_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyrcn_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyrcn_seq_elm_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

Next, we optimize ELMs from PyRCN using HAR as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. We aimed to optimize input scaling $\alpha_{\mathrm{u}}$, then bias scaling $\alpha_{\mathrm{bi}}$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyrcn_har_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4,
        'input_activation': 'tanh', 'bias_scaling': 0.0,
        'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("elm", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_elm_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'elm__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyrcn_har_seq_elm_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyrcn_har_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyrcn_har_seq_elm_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

## HP-ELM

Next, we optimize ELMs from HP-ELM using only the original time series as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. Note that the HP-ELM does not have the hyper-parameters input scaling $\alpha_{\mathrm{u}}$ and bias scaling $\alpha_{\mathrm{bi}}$. Hence we only jointly optimize the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/skelm_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {'n_neurons': 50, 'density': 0.1, 
                          'ufunc': 'tanh', 'alpha': 1e-5,
                          'random_state': 42}
    elm_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("elm", TransformedTargetRegressor(
                                       regressor=SkELMRegressor(**initial_elm_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__n_neurons':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/skelm_seq_elm_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/skelm_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/skelm_seq_elm_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

Next, we optimize ELMs from HP-ELM using HAR as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. Note that the HP-ELM does not have the hyper-parameters input scaling $\alpha_{\mathrm{u}}$ and bias scaling $\alpha_{\mathrm{bi}}$. Hence we only jointly optimize the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/skelm_har_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {'n_neurons': 200, 'density': 0.1,
                          'ufunc': 'tanh', 'alpha': 1e-5,
                          'random_state': 42}
    elm_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("elm", TransformedTargetRegressor(
                                       regressor=SkELMRegressor(**initial_elm_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__n_neurons':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/skelm_har_seq_elm_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/skelm_har_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/skelm_har_seq_elm_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

## PyESN ESN

Next, we optimize ESNs from PyESN using only the original time series as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. Note that PyESN does not have the hyper-parameters bias scaling $\alpha_{\mathrm{bi}}$ and leakage $\lambda$. Hence, we only jointly optimized input scaling $\alpha_{\mathrm{u}}$ and spectral radius $\rho$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyesn_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {'n_inputs': 1, 'n_outputs': 1, 'n_reservoir': 50,
                          'spectral_radius': 0.0, 'sparsity': 0.5,
                          'noise': 1e-3, 'input_shift': 0,
                          'input_scaling': 0.4, 'teacher_forcing': True,
                          'teacher_scaling': 1., 'teacher_shift': 0,
                          'random_state': 42}
    esn_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=PyESN(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__n_reservoir':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__noise': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyesn_seq_esn_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyesn_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyesn_seq_esn_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

Next, we optimize ESNs from PyESN using HAR as features.

In contrast to [[1]](#1), we did not jointly optimize all hyper-parameters but used our proposed sequential optimization. Note that PyESN does not have the hyper-parameters bias scaling $\alpha_{\mathrm{bi}}$ and leakage $\lambda$. Hence, we only jointly optimized input scaling $\alpha_{\mathrm{u}}$ and spectral radius $\rho$, and, finally, jointly the regularization parameter $\beta$ and the number of neurons $N_\mathrm{res}$.

In [ ]:
try:
    search = load(f'../results/pyesn_har_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {'n_inputs': 3, 'n_outputs': 1, 'n_reservoir': 50,
                          'spectral_radius': 0.0, 'sparsity': 0.5,
                          'noise': 1e-3, 'input_shift': 0,
                          'input_scaling': 0.4, 'teacher_forcing': True,
                          'teacher_scaling': 1., 'teacher_shift': 0,
                          'random_state': 42}
    esn_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=PyESN(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__n_reservoir':
                        [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__noise': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 200, 'random_state': 42, 'verbose': 1,
                    'n_jobs': -1, 'scoring': scoring, "refit": "R2",
                    "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'../results/pyesn_har_seq_esn_h{H}.joblib')
print(search.all_best_params_)

We do an additional GridSearchCV without parameters for the final evaluation on the test set, because the object returns a mass of useful information.

In this step, no parameters are optimized anymore.

In [ ]:
try:
    search_test = load(f'../results/pyesn_har_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'../results/pyesn_har_seq_esn_h{H}_test.joblib')

In [ ]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

In [ ]:
sns.set_theme(context="paper", style="whitegrid", font="serif", font_scale=.8, rc={"lines.linewidth": 0.8})

In [ ]:
search1 = load(f"results/pyesn_har_seq_esn_h{H}.joblib")
df1 = pd.DataFrame(search1.cv_results_).rename(
    columns={"param_esn__regressor__n_reservoir": "param_esn__regressor__hidden_layer_size"})
df1["Architecture"] = "PyESN"

search2 = load(f"results/pyrcn_har_seq_esn_h{H}.joblib")
df2 = pd.DataFrame(search2.cv_results_)
df2["Architecture"] = "PyRCN"
df = pd.concat([df1, df2]).reset_index(drop=True)

In [ ]:
import numpy as np
np.mean(df1.groupby(by="param_esn__regressor__hidden_layer_size").mean()["mean_score_time"] / df2.groupby(by="param_esn__regressor__hidden_layer_size").mean()["mean_score_time"])

In [ ]:
df1.groupby(by="param_esn__regressor__hidden_layer_size").mean()

In [ ]:
df2.groupby(by="param_esn__regressor__hidden_layer_size").mean()

TODO: Update the mean speedup

In [ ]:
fig, axs = plt.subplots()
fig.set_size_inches(4/1.6, 2.5/1.6)
sns.pointplot(data=df, x="param_esn__regressor__hidden_layer_size", y="mean_fit_time", hue="Architecture", linestyles=[":", ":"], markers="X", ci="sd", ax=axs)
sns.pointplot(data=df, x="param_esn__regressor__hidden_layer_size", y="mean_score_time", hue="Architecture", ci="sd", ax=axs)
axs.legend(handles=axs.lines[::8+1], labels=["PyESN fit time", "PyESN score time", "PyRCN fit time", "PyRCN score time"],
           loc='upper left', bbox_to_anchor=(0.0, 1.0), ncol=1)
axs.set_yscale('log')
axs.set(xlabel="Reservoir size", ylabel="Time in seconds")
plt.xticks(rotation=45)
plt.savefig('ESN_Benchmark.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
search1 = load(f"results/skelm_har_seq_elm_h{H}.joblib")
df1 = pd.DataFrame(search1.cv_results_).rename(
    columns={"param_elm__regressor__n_neurons": "param_elm__regressor__hidden_layer_size"})
df1["Architecture"] = "HP-ELM"

search2 = load(f"results/pyrcn_har_seq_elm_h{H}.joblib")
df2 = pd.DataFrame(search2.cv_results_)
df2["Architecture"] = "PyRCN"
df = pd.concat([df1, df2]).reset_index(drop=True)

In [ ]:
fig, axs = plt.subplots()
fig.set_size_inches(4/1.6, 2.5/1.6)
sns.pointplot(data=df, x="param_elm__regressor__hidden_layer_size", y="mean_fit_time", hue="Architecture", linestyles=[":", ":"], markers="X", ci="sd", ax=axs)
sns.pointplot(data=df, x="param_elm__regressor__hidden_layer_size", y="mean_score_time", hue="Architecture", ci="sd", ax=axs)
axs.legend(handles=axs.lines[::8+1], labels=["HP-ELM fit time", "HP-ELM score time", "PyRCN fit time", "PyRCN score time"],
           loc='upper left', bbox_to_anchor=(0.0, 1.0), ncol=1)
axs.set_yscale('log')
axs.set(xlabel="Reservoir size", ylabel="Time in seconds")
plt.xticks(rotation=45)
plt.savefig('ELM_Benchmark.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df1 = df1.sort_values(by="param_elm__regressor__hidden_layer_size").reset_index(drop=True)
df2 = df2.sort_values(by="param_elm__regressor__hidden_layer_size").reset_index(drop=True)
df1["fit_speedup"] = df1["mean_fit_time"] / df2["mean_fit_time"]
df1["score_speedup"] = df1["mean_score_time"] / df2["mean_score_time"]

In [ ]:
print(df1["params"][-1:], df2["params"][-1:])

In [ ]:
fig, axs = plt.subplots()
# fig.set_size_inches(4/1.6, 2.5/1.6)
sns.pointplot(data=df1, x="param_elm__regressor__hidden_layer_size", y="fit_speedup", linestyles=":", markers="X", ci="sd", ax=axs)
sns.pointplot(data=df1, x="param_elm__regressor__hidden_layer_size", y="score_speedup", ci="sd", ax=axs)
axs.legend(handles=axs.lines[::8+1], labels=["fit time", "PyESN score time", "PyRCN fit time", "PyRCN score time"],
           loc='upper left', bbox_to_anchor=(0.0, 1.0), ncol=1)
axs.set_yscale('log')
axs.set(xlabel="Reservoir size", ylabel="Time in seconds")
plt.xticks(rotation=45)
plt.savefig('ESN_speedup.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
import numpy as np
np.mean(df1["fit_speedup"][:31])

In [ ]:
df1[["param_elm__regressor__hidden_layer_size", "mean_fit_time", "mean_score_time"]][-31:]

In [ ]:
df2[["param_elm__regressor__hidden_layer_size", "mean_fit_time", "mean_score_time"]][-31:]

<a id="1">[1]</a> 
Gabriel Trierweiler Ribeiro, André Alves Portela Santos, Viviana Cocco Mariani, Leandro dos Santos Coelho. (2021). 
Novel hybrid model based on echo state neural network applied to the prediction of stock price return volatility. 
Expert Systems with Applications, 184, 115490. 
[10.1016/j.eswa.2021.115490](https://doi.org/10.1016/j.eswa.2021.115490)

<a id="2">[2]</a> 
Anton Akusok, Kaj-Mikael Björk, Yoan Miche, Amaury Lendasse. (2015). 
High-Performance Extreme Learning Machines: A Complete Toolbox for Big Data Applications.
Access, IEEE, 3, 1011-1025. 
[10.1109/ACCESS.2015.2450498](https://doi.org/10.1109/ACCESS.2015.2450498)

<a id="3">[3]</a> 
Fulvio Corsi. (2009). 
A Simple Approximate Long-Memory Model of Realized Volatility.
Journal of Financial Econometrics, 7, 174-196. 
[10.1093/jjfinec/nbp001](https://doi.org/10.1093/jjfinec/nbp001)